In [ ]:
# pip install pyspark
# pip install findspark

In [17]:
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

findspark.init()

def start_spark():
    local = "local[*]"
    appName = "VelibParis"
    localConfig = SparkConf().setAppName(appName).setMaster(local).\
    set("spark.executor.memory", "2G").\
    set("spark.driver.memory", "1G").\
    set("spark.sql.catalogImplementation", "in-memory")
    
    spark = SparkSession.builder.config(conf = localConfig).getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("ERROR")
    
    spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
    spark.conf.set("spark.sql.shuffle.partitions", "4")
    return spark

spark = start_spark()
print("Spark started")

Spark started


In [18]:
import urllib.request
import json

urlVelib = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json"

with urllib.request.urlopen(urlVelib) as response:
    encoding = response.info().get_content_charset('utf-8')
    obj = json.loads(response.read().decode(encoding))
    
rdd = spark.sparkContext.parallelize([json.dumps(obj)])
data = spark.read.json(rdd)
#data.printSchema()

stations = data.select(explode("data.stations").alias("stations"))
stations = stations.select("stations.station_id", col("stations.name").alias("Velib station"), col("stations.capacity").alias("Capacity"))

#stations.persist()
stations.show(truncate=False)

+----------+--------------------------------------+--------+
|station_id|Velib station                         |Capacity|
+----------+--------------------------------------+--------+
|213688169 |Benjamin Godard - Victor Hugo         |35      |
|653222953 |Mairie de Rosny-sous-Bois             |30      |
|516709288 |Charonne - Robert et Sonia Delauney   |20      |
|36255     |Toudouze - Clauzel                    |21      |
|37815204  |Mairie du 12ème                       |30      |
|100769544 |Harpe - Saint-Germain                 |45      |
|85002689  |Jourdan - Stade Charléty              |60      |
|123095125 |Alibert - Jemmapes                    |60      |
|120827885 |Messine - Place Du Pérou              |12      |
|251039991 |Cassini - Denfert-Rochereau           |25      |
|37874517  |Saint-Sulpice                         |21      |
|66491398  |Le Brun - Gobelins                    |48      |
|210403080 |Lacépède - Monge                      |23      |
|210561800 |Saint-Romain

In [20]:
station = "Nation"

urlVelib = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json"
with urllib.request.urlopen(urlVelib) as response:
    encoding = response.info().get_content_charset('utf-8')
    obj = json.loads(response.read().decode(encoding))
    
rdd = spark.sparkContext.parallelize([json.dumps(obj)])
data1 = spark.read.json(rdd)
#data1.printSchema()

data2 = data1.select(explode("data.stations").alias("data"))
data2 = data2.select("data.station_id", col("data.numBikesAvailable").alias("Available"))

data3 = stations.join(data2, "station_id").filter(col("Velib station").contains(station))
data3.show(truncate=False)

+-----------+-----------------------------------------+--------+---------+
|station_id |Velib station                            |Capacity|Available|
+-----------+-----------------------------------------+--------+---------+
|30730      |Place de la Nation - Charonne            |36      |7        |
|15153595   |Nationale - Vincent Auriol               |33      |9        |
|27415169   |Nationale - Marcel Duchamp               |23      |14       |
|54000570   |Place de la Nation - Picpus              |36      |6        |
|74635382   |Place de la Nation - Taillebourg         |47      |4        |
|129132870  |Nationale - Place Jules Guesde           |23      |7        |
|210568512  |Assemblée Nationale                      |63      |16       |
|581480367  |Fabre d'Eglantine - Place de la Nation   |68      |11       |
|653159330  |Quai de Sèvres - Manufacture Nationale   |28      |12       |
|1057302263 |Place Nationale - Château-des-Rentiers   |18      |2        |
|1062685145 |Bibliothèque